# Clustering

The goal of this program is to visualize and cluster the data.

## Input
* Importing the libaries that are necessary:

In [9]:
import numpy as np

* Importing the necessary files:
 * Read files
 * Convert read files to data
 * Save data in dataframes

In [2]:
def lib(FileName):
    def openFile(FileName):
        read_data = open(f'{FileName}')
        data = read_data.read()
        data = data.splitlines()
        read_data.close()
        
        return data
        
    lib = {}
    
    for lines in openFile(FileName):
        line = lines.split()
        lib[line[0]] = line[1:]
            
    return lib

lib_data = lib('Data\Voorbeeld_clusterdata.txt')
lib_results = lib('Data\Voorbeeld_clusterresult.txt')

# Own cluster method = gridbased clustering:

Om grids te maken zal elk coordinaat worden opgesplitst in een j aantal sub-cellen. Vervolgens zal voor elk coordinaat beslist worden in welke sub-cel deze past. Dit zal dus een getal zijn in de range van 1 tot j. Als elke sub-cel is bepaald ontstaat er een unieke code van sub-cellen. Voor elk van codes wordt er gekeken of er aanliggende sub-cellen zijn met datapoints er in. Als dit zo is zullen deze sub-cellen dezelfde groep worden.

## Stap 1: Productie van cellen:
### Stap 1.1:Productie intervallen per coordinaat:
Om de sub-cel groepen te maken, zullen er intervallen worden opgesteld:
1. Ondergrens = minimale waarde van het coordinaat-type
2. Bovengrens = maximale waarde van het coordinaat-type
3. J = het aantal intervallen dat geproduceerd moet worden

### Stap 1.2: Productie van cellen:
Om cellen te maken zullen alle sub-cellen van een sample bij elkaar worden gevoegd. Dit gebeurd als volgt:
1. Bepalen in welke sub-cel elk coordinaat van een sample hoort
2. sub-cellen samenvoegen tot 1 lange unieke code. Dit staat gelijk aan het cel-ID

In [3]:
def make_cells_from_lib(data,j=10):
    IDs = list(data.keys())
    
    def cords_list(data):
        cordinates = dict()
        
        for cor in range(len(data[list(data.keys())[0]])):
            x = list()
            for i in data:
                x.append(float(data[i][cor]))
                
            cordinates[cor] = tuple(x)
            
        return cordinates
        
    def make_grid(data,j):
        dic = dict()
        
        for i in cords:
            minim = np.min(cords[i])
            maxim = np.max(cords[i])
            
            dic[i] = np.mgrid[minim:maxim:complex(0,j)]
            
        return dic
    
    cords = cords_list(data)
    intervals = make_grid(cords,j)
            
    
    labels = list()
    for i in cords:
        x_inter_label = list()
        for j in range(1,len(intervals[i])):
            vals = list((cords[i] >= intervals[i][j-1]) & (cords[i] < intervals[i][j]))
            vals = np.where(vals==False, 0, vals)
            vals = np.where(vals==True, j, vals)       
                        
            x_inter_label.append(vals)
        
        labels.append(x_inter_label)
        
        
    labelzz = list()
    for i in range(len(labels)):
        for k in range(1,len(labels[i])):
            labels[i][0] = np.add(labels[i][0],labels[i][k])
        labelzz.append(labels[i][0])
        
    del labels
    
    lib_cells = dict()     
    
    for i in range(len(labelzz[0])):
        cell_id = list()
        for j in range(len(labelzz)):
            cell_id.append(labelzz[j][i])
            
        lib_cells[IDs[i]] = cell_id
        
    return lib_cells
        
lib_cells = make_cells_from_lib(lib_data)

In [4]:
lib_cells

{'846160': [7, 6, 7, 7, 6, 8, 8, 8],
 '820434': [6, 5, 6, 7, 5, 6, 7, 7],
 '849103': [6, 4, 6, 7, 5, 7, 7, 7],
 '846353': [8, 6, 7, 8, 7, 8, 8, 8],
 '848613': [6, 5, 6, 7, 5, 7, 7, 7],
 '846978': [8, 7, 7, 8, 7, 8, 8, 9],
 '847332': [7, 5, 6, 7, 4, 6, 7, 7],
 '847300': [7, 5, 6, 6, 2, 3, 6, 5],
 '846924': [7, 6, 7, 8, 6, 8, 8, 8],
 '846452': [7, 6, 7, 7, 7, 7, 8, 8],
 '818743': [8, 7, 8, 8, 7, 8, 8, 9],
 '1005857': [7, 6, 6, 7, 6, 7, 8, 8],
 '1005802': [7, 5, 6, 7, 5, 6, 7, 7],
 '1005968': [8, 7, 8, 8, 7, 8, 8, 8],
 '1005901': [7, 6, 7, 8, 7, 7, 8, 8],
 '1005990': [8, 7, 7, 8, 7, 8, 8, 8],
 '1005953': [8, 7, 8, 8, 7, 8, 8, 9],
 '1005899': [6, 4, 5, 6, 5, 6, 7, 7],
 '620243': [7, 5, 6, 6, 5, 6, 6, 6],
 '1005717': [8, 7, 8, 8, 8, 8, 8, 8],
 '1005837': [5, 3, 4, 6, 4, 5, 6, 6],
 '521046': [7, 6, 8, 8, 5, 7, 7, 7],
 '518990': [8, 6, 7, 7, 7, 7, 8, 8],
 '518979': [7, 6, 8, 8, 5, 7, 8, 7],
 '518988': [5, 4, 5, 6, 5, 6, 7, 7],
 '518896': [7, 6, 7, 8, 7, 8, 8, 8],
 '521019': [7, 6, 7, 8, 7, 8,

## Stap 2: Productie van labels:

### Stap 2.1: Achterhalen wat buren zijn:
Om te achterhalen of een cel buren heeft wordt er van elke cel de afstand tot alle andere mogelijke cellen uitgerekend. Dit gebeurd doormiddel van:
    
>   `np.substract(orginele_cel, elke_andere_cel)`

Hierkomt het volgende uit:

>    `[0,2,...,1,0]`

Als elke waarde in de lijst dat ontstaat binnen de range `r` valt wordt het ID van de cel opgeslagen in een lijst en wordt deze lijst gereturned. 

### Stap 2.2: Buren clusteren:
Om vervolgens clusters toe te kennen aan de buren, moeten alle mogelijke buren van een originele cel worden bepaald. Als alle buren zijn vast gesteld worden zij in een `dict()` opgeslagen met het clusternummer als key en elk eiwit-ID als value.

In [5]:
def get_labels(data):
    def get_neigb_id(ID,r=1):
        neigb = list()
        for indx in lib_cells:
            distance = np.subtract(lib_cells[ID],lib_cells[indx],out=np.array([0 for i in range(8)]))
            if all([-r < i < r for i in distance]):
                neigb.append(indx)

        return neigb

    indexes = [id for id in data]
    new_indexes = indexes.copy()
    
    def make_groups(data_point):
        neigbors = get_neigb_id(data_point)
        new_neigbors = list()
        
        for cel in neigbors:
            new_cells = get_neigb_id(cel)
            
            for new_cel in new_cells:
                if new_cel not in neigbors:
                    new_neigbors.append(new_cel)
                    print(new_cel)
                    
            neigbors += new_neigbors
            
        for neigb in neigbors:
            new_indexes.remove(neigb)
            
        return neigbors
    
    options = list()
    
    for index in new_indexes:
        options.append(make_groups(index))
    
    
    labeled_data = dict()
    
    tresh = 4
    cluster = 0
    
    for i in range(len(options)):
        if len(options[i]) > tresh:
            labeled_data[cluster] = options[i]
            cluster += 1
        
    return labeled_data
    
results = get_labels(lib_cells)

In [6]:
results

{0: ['846160',
  '1382381',
  '657946',
  '350787',
  '1109733',
  '819970',
  '1511542',
  '681768',
  '791607',
  '599134'],
 1: ['849103',
  '533961',
  '1077529',
  '831853',
  '368082',
  '1195684',
  '1177854',
  '516294',
  '517227',
  '1066981',
  '1078999',
  '634781',
  '317328',
  '860885',
  '1148836',
  '1148798',
  '1260230',
  '865166',
  '681236',
  '481146'],
 2: ['848613',
  '963217',
  '389562',
  '1227964',
  '831751',
  '367133',
  '2101885',
  '1067050',
  '1065847',
  '1066710',
  '596813',
  '598072',
  '1278753',
  '876696',
  '819760',
  '819928',
  '540946',
  '516471',
  '1053922',
  '1002668',
  '777834',
  '871756',
  '891219',
  '1038416',
  '1037686',
  '522368',
  '481915',
  '653231',
  '1497623',
  '764568',
  '422226',
  '907486',
  '865426',
  '874189',
  '875572',
  '1038042',
  '635274',
  '1148891',
  '1244627',
  '1478461',
  '946187',
  '599101',
  '934892',
  '617627',
  '642394',
  '622383',
  '598060',
  '598146',
  '481507',
  '456249',
  '

# Productie van labels, maar dan efficienter

In [7]:
def get_neigb_id(ID,r=1):
    neigb = list()
    for indx in lib_cells:
        distance = np.subtract(lib_cells[ID],lib_cells[indx],out=np.array([0 for i in range(8)]))
        if all([-r < i < r for i in distance]):
            neigb.append(indx)

    return neigb
    
indexes = [idx for idx in lib_cells]




847332
1179412
1229766
573770
